In [1]:
# Notebook for making the figure for the top of the article
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)

plt.style.use('ggplot')
plt.style.use('fivethirtyeight')

fs_label = 24
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[math.floor(meanWidth)-1:]

saveFigures is set to: True
Done loading packages


In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd()[:-5] + ssidatapath
dfAdm = pd.read_excel('..\Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')
diffDates = curDates[1:]
dfCase = pd.DataFrame(columns=dfAdm.columns)

In [3]:
for k in range(0,len(curDates)):
    d = curDates[k]
    curIndex = dfAdm.index[k]

    thisDateStr = d.strftime('%Y-%m-%d')
    curDir = rootdir + '/SSI_data_' + thisDateStr 
    curFilePath = curDir + '/Cases_by_age.csv'
    casedatadf = pd.read_csv(curFilePath,delimiter=';',dtype=str)

    casedatadf['Antal_bekræftede_COVID-19'] = pd.to_numeric(casedatadf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))

    dfCase.loc[curIndex] = casedatadf['Antal_bekræftede_COVID-19'].values

In [12]:
# Same things, but with cases
dfCaseDiff = dfCase.diff().iloc[1:]

dfCaseDiff['Dato'] = curDates[1:]

# Bad hack to ignore 10-15 year olds!
dfCaseDiff['10-19'] = np.multiply(dfCaseDiff['10-19'],0.4)

dfCaseDiffSum = dfCaseDiff.iloc[:,1:-2].cumsum(axis=1)
dfCaseDiffSum['Dato'] = curDates[1:]

# dfCaseDiff = dfCaseDiff.iloc[:-1]
# dfCaseDiffSum = dfCaseDiffSum.iloc[:-1]


dfCaseDiffSum['I alt'] = dfCaseDiffSum['90+']
dfCaseDiffSum.tail()

Aldersgruppe,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Dato,I alt
04_05_2021,390.0,1594.0,2430.0,3257.0,3956.0,4328.0,4448.0,4464.0,4470.0,2021-05-04,4470.0
11_05_2021,538.8,1954.8,2896.8,3822.8,4608.8,4985.8,5092.8,5118.8,5119.8,2021-05-11,5119.8
18_05_2021,686.8,2434.8,3433.8,4442.8,5265.8,5619.8,5718.8,5735.8,5742.8,2021-05-18,5742.8
25_05_2021,616.0,2496.0,3379.0,4373.0,5190.0,5509.0,5574.0,5579.0,5580.0,2021-05-25,5580.0
01_06_2021,580.0,2569.0,3468.0,4473.0,5281.0,5526.0,5567.0,5587.0,5591.0,2021-06-01,5591.0


# Make figure

In [19]:

meanWidth=3
rnMeanData = rnMean(dfCaseDiffSum['I alt'],meanWidth)
rnTimeDates = rnTime(diffDates,meanWidth)

# predDates = rnTimeDates[-1]
# predDates = (diffDates[-1] + np.arange(np.timedelta64(-7*2,'D'),np.timedelta64(7*6,'D'),np.timedelta64(7,'D'))).astype('datetime64[D]')
# predDates

predValsInit = rnMeanData[-1]

weekPred = 10

# predDates = np.zeros((weekPred,1))
predVals = np.zeros((weekPred,1))

curDate = rnTimeDates[-1]
curVal1 = predValsInit
curVal2 = predValsInit
curVal3 = predValsInit
# predDates = np.array(curDate)
predDates = []
predVals1 = []
predVals2 = []
predVals3 = []
# predVals[0] = predValsInit

for k in range(0,weekPred+1):

    
    predDates.append(curDate)
    predVals1.append(curVal1)
    predVals2.append(curVal2)
    predVals3.append(curVal3)


    curDate = curDate + np.timedelta64(7,'D')
    curRT = 0.95
    curVal1 = curVal1 * (curRT**(4.7/7))
    curRT = 1.1
    curVal2 = curVal2 * (curRT**(4.7/7))
    curRT = 1.3
    curVal3 = curVal3 * (curRT**(4.7/7))


# predDates
# predVals

DatetimeIndex(['2020-11-17', '2020-11-24', '2020-12-01', '2020-12-08',
               '2020-12-15', '2020-12-22', '2020-12-29', '2021-01-05',
               '2021-01-12', '2021-01-19', '2021-01-26', '2021-02-02',
               '2021-02-09', '2021-02-16', '2021-02-23', '2021-03-02',
               '2021-03-09', '2021-03-15', '2021-03-23', '2021-03-30',
               '2021-04-06', '2021-04-13', '2021-04-20', '2021-04-27',
               '2021-05-04', '2021-05-11', '2021-05-18', '2021-05-25',
               '2021-06-01'],
              dtype='datetime64[ns]', freq=None)

In [6]:
fig,ax1 = plt.subplots()

ax1.plot(diffDates,dfCaseDiffSum['I alt'],'k.:',markersize=14,linewidth=0.5)


for k in range(0,len(predDates)):
    ax1.plot(predDates[:k],predVals1[:k],color=(0,0,0,0.05))
    ax1.plot(predDates[:k],predVals2[:k],color=(0,0,0,0.05))
    ax1.plot(predDates[:k],predVals3[:k],color=(0,0,0,0.05))



# for k in range(0,len(predDates)):
#     ax1.plot(predDates[:k],predVals1[:k],color=(0,0.2,0,0.05))
#     ax1.plot(predDates[:k],predVals2[:k],color=(0,0,0.5,0.05))
#     ax1.plot(predDates[:k],predVals3[:k],color=(0.4,0,0,0.05))


# ax1.plot(predDates,predVals2)
# ax1.plot(predDates,predVals3)

ax1.plot(rnTimeDates,rnMeanData,'k',linewidth=4)


ax1.text(np.datetime64('2021-08-01'),5500,'?',fontsize=70)



ax1.text(np.datetime64('2021-07-05'),8500,
    '- Genåbning \n- Adfærdsændringer \n- Kørekort erstatter pas \n- Indisk variant \n- ...',
    fontsize=16)
ax1.annotate('', xy=(np.datetime64('2021-06-28'),11000), xytext=(np.datetime64('2021-06-28'),8500),
            arrowprops=dict(facecolor='black'))

ax1.text(np.datetime64('2021-07-05'),1100,
    '- Flere vaccinerede \n- Enorm testindsats \n- Sæsoneffekt \n- Tidligere smittede \n- ...',
    fontsize=16)
ax1.annotate('', xy=(np.datetime64('2021-06-28'),1000), xytext=(np.datetime64('2021-06-28'),3500),
            arrowprops=dict(facecolor='black'))


# right_side = ax1.spines["right"]
# right_side.set_visible(False)
# top_side = ax1.spines["top"]
# top_side.set_visible(False)

ax1.set_ylim([0,13000])
ax1.set_xlim([np.datetime64('2021-03-01'),np.datetime64('2021-09-01')])

ax1.set_ylabel('Nye smittetilfælde om ugen',color='black')

plt.xticks(rotation=45,fontsize=18)
plt.yticks(fontsize=22)

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%B'))

plt.tight_layout()

plt.savefig('SommerFremskrivningTitelFigur')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
fig,ax1 = plt.subplots()

ax1.plot(diffDates,dfCaseDiffSum['I alt'],'b.:',markersize=14,linewidth=0.5)


# for k in range(0,len(predDates)):
#     ax1.plot(predDates[:k],predVals1[:k],color=(0,0,0,0.05))
#     ax1.plot(predDates[:k],predVals2[:k],color=(0,0,0,0.05))
#     ax1.plot(predDates[:k],predVals3[:k],color=(0,0,0,0.05))



for k in range(0,len(predDates)):
    ax1.plot(predDates[:k],predVals1[:k],color=(0,0.2,0,0.05))
    ax1.plot(predDates[:k],predVals2[:k],color=(0,0,0,0.05))
    ax1.plot(predDates[:k],predVals3[:k],color=(0.4,0,0,0.05))


# ax1.plot(predDates,predVals2)
# ax1.plot(predDates,predVals3)

ax1.plot(rnTimeDates,rnMeanData,linewidth=4,color='darkblue')


ax1.text(np.datetime64('2021-08-01'),5500,'?',fontsize=70)



ax1.text(np.datetime64('2021-07-05'),8500,
    '- Genåbning \n- Adfærdsændringer \n- Kørekort erstatter pas \n- Indisk variant \n- ...',
    fontsize=16)
ax1.annotate('', xy=(np.datetime64('2021-06-28'),11000), xytext=(np.datetime64('2021-06-28'),8500),
            arrowprops=dict(facecolor='black'))

ax1.text(np.datetime64('2021-07-05'),1100,
    '- Flere vaccinerede \n- Enorm testindsats \n- Sæsoneffekt \n- Tidligere smittede \n- ...',
    fontsize=16)
ax1.annotate('', xy=(np.datetime64('2021-06-28'),1000), xytext=(np.datetime64('2021-06-28'),3500),
            arrowprops=dict(facecolor='black'))


# right_side = ax1.spines["right"]
# right_side.set_visible(False)
# top_side = ax1.spines["top"]
# top_side.set_visible(False)

ax1.set_ylim([0,13000])
ax1.set_xlim([np.datetime64('2021-03-01'),np.datetime64('2021-09-01')])

ax1.set_ylabel('Nye smittetilfælde om ugen',color='black')

plt.xticks(rotation=45,fontsize=18)

plt.yticks(fontsize=22)

for spine in ax1.spines.values():
    spine.set_edgecolor('xkcd:pale blue')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%B'))

# fig.patch.set_facecolor('xkcd:pale blue')
# plt.rcParams['axes.facecolor'] = 'none'
ax1.set_facecolor('xkcd:pale blue')

plt.tight_layout()

plt.savefig('SommerFremskrivningTitelFigurFarve',facecolor="xkcd:pale blue")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
fig,ax1 = plt.subplots()

ax1.plot(diffDates,dfCaseDiffSum['I alt'],'b.:',markersize=14,linewidth=0.5)


# for k in range(0,len(predDates)):
#     ax1.plot(predDates[:k],predVals1[:k],color=(0,0,0,0.05))
#     ax1.plot(predDates[:k],predVals2[:k],color=(0,0,0,0.05))
#     ax1.plot(predDates[:k],predVals3[:k],color=(0,0,0,0.05))



for k in range(0,len(predDates)):
    ax1.plot(predDates[:k],predVals1[:k],color=(0,0.2,0,0.05))
    ax1.plot(predDates[:k],predVals2[:k],color=(0,0,0,0.05))
    ax1.plot(predDates[:k],predVals3[:k],color=(0.4,0,0,0.05))


# ax1.plot(predDates,predVals2)
# ax1.plot(predDates,predVals3)

ax1.plot(rnTimeDates,rnMeanData,linewidth=4,color='darkblue')


ax1.text(np.datetime64('2021-07-20'),5000,'?',fontsize=140)



ax1.text(np.datetime64('2021-07-05'),8500,
    'Genåbning \nAdfærd \nVarianter \n...',
    fontsize=26)
ax1.annotate('', xy=(np.datetime64('2021-06-28'),12000), xytext=(np.datetime64('2021-06-28'),8000),
            arrowprops=dict(facecolor='black',width=7))

ax1.text(np.datetime64('2021-07-05'),1100,
    'Vaccine \nTest \nSæson \n...',
    fontsize=26)
ax1.annotate('', xy=(np.datetime64('2021-06-28'),500), xytext=(np.datetime64('2021-06-28'),4500),
            arrowprops=dict(facecolor='black',width=7))


# right_side = ax1.spines["right"]
# right_side.set_visible(False)
# top_side = ax1.spines["top"]
# top_side.set_visible(False)

ax1.set_ylim([0,13000])
ax1.set_xlim([np.datetime64('2021-03-01'),np.datetime64('2021-09-01')])

ax1.set_ylabel('Nye smittetilfælde om ugen',color='black')

plt.xticks(rotation=45,fontsize=18)

plt.yticks(fontsize=22)

for spine in ax1.spines.values():
    spine.set_edgecolor('xkcd:pale blue')

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%B'))

fig.patch.set_facecolor('xkcd:pale blue')
# plt.rcParams['axes.facecolor'] = 'none'
ax1.set_facecolor('xkcd:pale blue')

plt.tight_layout()

plt.savefig('SommerFremskrivningTitelFigur_V2_Farve',facecolor="xkcd:pale blue")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …